In [6]:
import sys
import os
from imp import reload
from image import utils
import numpy as np
import utils; reload(utils)

<module 'utils' from '../../ml_utils/image/utils.py'>

In [5]:
!pip install bcolz

    100% |████████████████████████████████| 1.3MB 1.1MB/s eta 0:00:01
  Running setup.py bdist_wheel for bcolz ... done
  Stored in directory: /root/.cache/pip/wheels/e9/84/eb/f8f3caa627bb01ebc96034c3411f59870951246e5873b3f4c7
Successfully built bcolz


In [4]:
%load_ext autoreload
%autoreload 2

In [10]:
path = r'/data/'
batch_size=64
# os.mkdir(path+'results')

In [2]:
import sys
sys.path.insert(0, '../../ml_utils')
sys.path.insert(0, '../../ml_utils/image')

In [12]:
!mkdir /results

mkdir: cannot create directory ‘/data/results’: Read-only file system


In [15]:
# trn = utils.get_data(path + r'raw/trn')
# val = utils.get_data(path + r'raw/valid')
# test = utils.get_data(path + r'test')


In [14]:
utils.save_array('/output/trn.dat', trn)
utils.save_array('/output/val.dat', val)
utils.save_array('/output/test.dat', test)

In [16]:
(val_classes, trn_classes, val_labels, trn_labels, 
    val_filenames, filenames, test_filenames) = utils.get_classes(path)

Found 9403 images belonging to 120 classes.
Found 819 images belonging to 120 classes.
Found 10357 images belonging to 1 classes.


In [17]:
raw_filenames = [f.split('/')[-1] for f in filenames]
raw_test_filenames = [f.split('/')[-1] for f in test_filenames]
raw_val_filenames = [f.split('/')[-1] for f in val_filenames]

In [63]:
# os.mkdir(path+'sample')
# os.mkdir(path+'sample/train')
# os.mkdir(path+'sample/valid')

# from shutil import copyfile
# from glob import glob

# g = glob(path+'raw/trn/*')
# for d in g: 
#     d = d.split('/')[-1]
#     os.mkdir(path+'sample/train/'+d)
#     os.mkdir(path+'sample/valid/'+d)
# g = glob(path+'raw/trn/*/*.jpg')
# shuf = np.random.permutation(g)
# for i in range(400): 
#     copyfile(shuf[i], path+'sample/train/' + shuf[i].split('/')[-1])
# g = glob(path+'raw/valid/*/*.jpg')
# shuf = np.random.permutation(g)
# for i in range(200): 
#     copyfile(shuf[i], path+'sample/valid/' + shuf[i].split('/')[-1])

In [21]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.layers import BatchNormalization, Convolution2D, MaxPool2D, Dropout, GlobalAveragePooling2D, Activation
from keras.layers import MaxPooling2D
from keras.models import Sequential
import numpy as np

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))

In [22]:
conv_feat = base_model.predict(trn)
conv_val_feat = base_model.predict(val)
conv_test_feat = base_model.predict(test)

model.layers

In [23]:
utils.save_array('/output/conv_feat.dat', conv_feat)
utils.save_array('/output/conv_val_feat.dat', conv_val_feat)
utils.save_array('/output/conv_test_feat.dat', conv_test_feat)

In [25]:
conv_val_feat.shape

(819, 1, 1, 2048)

In [34]:
from keras.layers import MaxPool2D, BatchNormalization, Flatten, Dense
from keras.models import Sequential
from keras.optimizers import Adam
def get_bn_layers(p,input_layers):
    return [
        BatchNormalization(input_shape=input_layers[-1].output_shape[1:], axis=1),
        Dropout(p/4),
        Flatten(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(p/2),
        Dense(120, activation='softmax')
    ]
bn_model = Sequential(get_bn_layers(0.5, base_model.layers))
# bn_model = Sequential(get_bn_layers(p))
bn_model.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
bn_model.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [38]:
bn_model.fit(conv_feat, trn_labels, batch_size=batch_size, epochs=20, 
             validation_data=(conv_val_feat, val_labels))

Train on 9403 samples, validate on 819 samples
Epoch 1/20
9403/9403 [==============================] - 1s - loss: 0.7580 - acc: 0.7604 - val_loss: 1.6013 - val_acc: 0.5897
Epoch 2/20
9403/9403 [==============================] - 1s - loss: 0.7109 - acc: 0.7765 - val_loss: 1.5796 - val_acc: 0.5849
Epoch 3/20
9403/9403 [==============================] - 1s - loss: 0.6500 - acc: 0.7957 - val_loss: 1.6253 - val_acc: 0.5788
Epoch 4/20
9403/9403 [==============================] - 1s - loss: 0.5910 - acc: 0.8104 - val_loss: 1.6093 - val_acc: 0.5885
Epoch 5/20
9403/9403 [==============================] - 1s - loss: 0.5508 - acc: 0.8237 - val_loss: 1.6002 - val_acc: 0.5922
Epoch 6/20
9403/9403 [==============================] - 1s - loss: 0.5309 - acc: 0.8321 - val_loss: 1.6733 - val_acc: 0.5885
Epoch 7/20
9403/9403 [==============================] - 1s - loss: 0.5119 - acc: 0.8343 - val_loss: 1.7158 - val_acc: 0.5824
Epoch 8/20
9403/9403 [==============================] - 1s - loss: 0.4758 - ac

In [39]:
bn_model.optimizer.lr = 1e-4


In [41]:
bn_model.fit(conv_feat, trn_labels, batch_size=batch_size, epochs=5, 
             validation_data=(conv_val_feat, val_labels))

Train on 9403 samples, validate on 819 samples
Epoch 1/5
9403/9403 [==============================] - 1s - loss: 0.2863 - acc: 0.9053 - val_loss: 1.9194 - val_acc: 0.5885
Epoch 2/5
9403/9403 [==============================] - 1s - loss: 0.2923 - acc: 0.9025 - val_loss: 1.9793 - val_acc: 0.5922
Epoch 3/5
9403/9403 [==============================] - 1s - loss: 0.2492 - acc: 0.9205 - val_loss: 1.9245 - val_acc: 0.5971
Epoch 4/5
9403/9403 [==============================] - 1s - loss: 0.2824 - acc: 0.9072 - val_loss: 2.0203 - val_acc: 0.5849
Epoch 5/5
9403/9403 [==============================] - 1s - loss: 0.2617 - acc: 0.9078 - val_loss: 1.9980 - val_acc: 0.5690


In [48]:
pred_test = bn_model.predict(conv_test_feat)

In [45]:
import pandas as pd


In [46]:
sub = pd.read_csv('/data/sample_submission.csv')

In [47]:
sub.head()

,id,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,...,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
0,000621fb3cbb32d8935728e48679680e,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
1,00102ee9d8eb90812350685311fe5890,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
2,0012a730dfa437f5f3613fb75efcd4ce,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
3,001510bc8570bbeee98c8d80c8a95ec1,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
4,001a5f3114548acdefa3d4da05474c2e,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333


In [54]:
utils.save_array('/output/results', pred_test)

In [57]:
import pickle as pkl

In [59]:
pkl.dump(pred_test, open('/output/result.pkl', 'wb'))